# Projet Final Apache Spark

Nom Etudiant :  **DIAW**

Prenom Etudiant:  **Abdoulaye Bara**

Classe :  **Big Data Analytics**


## Description
Ce projet consiste à utiliser Apache Spark pour faire l'analyse et le traitement des données de **[San Francisco Fire Department Calls ](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)** afin de fournir quelques KPI (*Key Performance Indicator*). Le **SF Fire Dataset** comprend les réponses aux appels de toutes les unités d'incendie. Chaque enregistrement comprend le numéro d'appel, le numéro d'incident, l'adresse, l'identifiant de l'unité, le type d'appel et la disposition. Tous les intervalles de temps pertinents sont également inclus. Étant donné que ce Dataset est basé sur les réponses et que la plupart des appels impliquent plusieurs unités, ainsi il existe plusieurs enregistrements pour chaque numéro d'appel. Les adresses sont associées à un numéro de bloc, à une intersection ou à une boîte d'appel, et non à une adresse spécifique.

**Plus de details sur la description des données cliquer sur ce [lien](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)**

## Travail à faire.
L'objectif de ce projet est de comprendre le **SF Fire Dataset** afin de bien répondre aux questions en utilisant les codes Spark/Scala adéquates.

- Créer un repos git (public) et partager le repos avec mon mail (limahin10@gmail.com)
- Ecrire un code lisible et bien indenté 
- N'oublier pas de mettre en commentaire la justification de vos réponses sur les cellules Markdown. 


## Note:
- Le projet est personnel, c'est-à-dire chaque notebook ne concerne qu'un seul étudiant. 
- Deadline : **Samedi 31 Juillet 2021  à 23h 59** (Aucune de dérogation ne sera acceptée)

### Chargement des données

Importation des packages Spark

In [1]:
import org.apache.spark.sql.types._ 
import org.apache.spark.sql.functions._ 
import spark.implicits._

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.37:4040
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1627746310135)
SparkSession available as 'spark'


import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import spark.implicits._


Nous allons jeter un coup d'oeil sur la structure des données avant de définir un schéma

In [2]:
!head -1 "datasets/sf-fire/sf-fire-calls.csv"

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,CallFinalDisposition,AvailableDtTm,Address,City,Zipcode,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumAlarms,UnitType,UnitSequenceInCallDispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhood,Location,RowID,Delay



Vu que la taille de ces données est énormes, inferer le schema pour un très grande volumes de données s'avère un peu couteux. Nous allons ainsi définir un schema pour le Dataset.

In [3]:
val fireSchema = StructType(Array(StructField("CallNumber", IntegerType, true),
  StructField("UnitID", StringType, true),
  StructField("IncidentNumber", IntegerType, true),
  StructField("CallType", StringType, true),                  
  StructField("CallDate", StringType, true),      
  StructField("WatchDate", StringType, true),
  StructField("CallFinalDisposition", StringType, true),
  StructField("AvailableDtTm", StringType, true),
  StructField("Address", StringType, true),       
  StructField("City", StringType, true),       
  StructField("Zipcode", IntegerType, true),       
  StructField("Battalion", StringType, true),                 
  StructField("StationArea", StringType, true),       
  StructField("Box", StringType, true),       
  StructField("OriginalPriority", StringType, true),       
  StructField("Priority", StringType, true),       
  StructField("FinalPriority", IntegerType, true),       
  StructField("ALSUnit", BooleanType, true),       
  StructField("CallTypeGroup", StringType, true),
  StructField("NumAlarms", IntegerType, true),
  StructField("UnitType", StringType, true),
  StructField("UnitSequenceInCallDispatch", IntegerType, true),
  StructField("FirePreventionDistrict", StringType, true),
  StructField("SupervisorDistrict", StringType, true),
  StructField("Neighborhood", StringType, true),
  StructField("Location", StringType, true),
  StructField("RowID", StringType, true),
  StructField("Delay", FloatType, true)))

fireSchema: org.apache.spark.sql.types.StructType = StructType(StructField(CallNumber,IntegerType,true), StructField(UnitID,StringType,true), StructField(IncidentNumber,IntegerType,true), StructField(CallType,StringType,true), StructField(CallDate,StringType,true), StructField(WatchDate,StringType,true), StructField(CallFinalDisposition,StringType,true), StructField(AvailableDtTm,StringType,true), StructField(Address,StringType,true), StructField(City,StringType,true), StructField(Zipcode,IntegerType,true), StructField(Battalion,StringType,true), StructField(StationArea,StringType,true), StructField(Box,StringType,true), StructField(OriginalPriority,StringType,true), StructField(Priority,StringType,true), StructField(FinalPriority,IntegerType,true), StructField(ALSUnit,BooleanType,true)...


In [4]:
val sfFireFile = "datasets/sf-fire/sf-fire-calls.csv"
val fireDF = spark
  .read
  .schema(fireSchema)
  .option("header", "true")
  .csv(sfFireFile)

sfFireFile: String = datasets/sf-fire/sf-fire-calls.csv
fireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


Nous allons mettre en cache le Dataframe

In [5]:
fireDF.cache()

res0: fireDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


In [6]:
fireDF.count()

res1: Long = 175296


In [7]:
fireDF.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

In [8]:
fireDF.show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

Filtrage des d'appels de type "Medical Incident"

In [9]:
val fewFireDF = fireDF
  .select("IncidentNumber", "AvailableDtTm", "CallType") 
  .where($"CallType" =!= "Medical Incident")

fewFireDF.show(5, false)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



fewFireDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [IncidentNumber: int, AvailableDtTm: string ... 1 more field]


### Question 1
**Combien de types d'appels distincts ont été passés ?**  
Pour être sûr, il ne faut pas compter les valeurs «nulles» dans la colonne.

#### Reponse 1
Nous utilisons la commande ***select()*** pour selectionner la colonne **CallType** du dataframe contenant le type des appels. La fonction ***na.drop()*** permet de supprimer les valeurs nulles de la colonne. La fonction ***distinct()*** supprime les doublons. La fonction ***count()*** permet de compter le nombre de ligne suivant les criteres precedents.

In [10]:
fireDF.select($"CallType").na.drop.distinct.count

res5: Long = 30


### Question 2

**Quels types d'appels différents ont été passés au service d'incendie?**

#### Reponse 2
Ici nous utilisons la fonction ***groupBy()*** pour regrouper les colonnes **CallType** et **FirePreventionDistrict**. Nous utilisons ensuite la fonction d'agregation ***count()*** pour appliquer un ***orderBy()*** dans la colonne **FirePreventionDistrict**. La fonction ***show()*** permet d'afficher le resultat de la liste de tous les types d'appels qui ont ete passes sur chaque service d'incendie.

In [11]:
fireDF.groupBy($"CallType", $"FirePreventionDistrict").count.orderBy($"FirePreventionDistrict").show(30, false)

+--------------------------------------------+----------------------+-----+
|CallType                                    |FirePreventionDistrict|count|
+--------------------------------------------+----------------------+-----+
|Vehicle Fire                                |1                     |55   |
|Marine Fire                                 |1                     |2    |
|Oil Spill                                   |1                     |4    |
|Explosion                                   |1                     |12   |
|Medical Incident                            |1                     |11302|
|Outside Fire                                |1                     |418  |
|Watercraft in Distress                      |1                     |9    |
|Extrication / Entrapped (Machinery, Vehicle)|1                     |2    |
|Structure Fire                              |1                     |2788 |
|Electrical Hazard                           |1                     |20   |
|Assist Poli

### Question 3

**Trouver toutes les réponses ou les délais sont supérieurs à 5 minutes?**

*Indication
1. Renommer la colonne Delay -> ReponseDelayedinMins
2. Retourner un nouveau DataFrame
3. Afficher tous les appels où le temps de réponse à un site d'incendie a eu lieu après un retard de plus de 5 minutes

#### Reponse 3 


In [12]:
// Reponse 3
val newFireDF = fireDF.withColumnRenamed("Delay", "ResponseDelayedinMins")
newFireDF

newFireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
res7: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


Avec la fonction ***select()*** nous selectionnons les colonnes **CallNumber** et **ResponseDelayedinMins**. Nous utilisons ensuite la fonction ***filter()*** pour pour premdre l'ensemble des reponses ayant un retard de plus de 5 minutes. 

In [13]:
newFireDF
    .select($"CallNumber", $"ResponseDelayedinMins")
    .filter($"ResponseDelayedinMins" > 5)
    .show(false)

+----------+---------------------+
|CallNumber|ResponseDelayedinMins|
+----------+---------------------+
|20110315  |5.35                 |
|20120147  |6.25                 |
|20130013  |5.2                  |
|20140067  |5.6                  |
|20140177  |7.25                 |
|20150056  |11.916667            |
|20150254  |5.116667             |
|20150265  |8.633333             |
|20150265  |95.28333             |
|20150380  |5.45                 |
|20150414  |7.6                  |
|20160059  |6.133333             |
|20160064  |5.1833334            |
|20170118  |6.9166665            |
|20170342  |5.2                  |
|20180129  |6.35                 |
|20180191  |7.983333             |
|20180382  |13.55                |
|20190062  |5.15                 |
|20190097  |13.583333            |
+----------+---------------------+
only showing top 20 rows



### Transformations des dates  
Maintenant nous allons d'abord:  
1. Transformer les dates de type String en Spark Timestamp afin que nous puissions effectuer des requêtes basées sur la date plus tard    
2. Retourner le Dataframe transformée  
3. Mettre en cache le nouveau DataFrame  

In [14]:
val fireTSDF = newFireDF
  .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy")).drop("CallDate") 
  .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy")).drop("WatchDate") 
  .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a")).drop("AvailableDtTm")

fireTSDF.cache()

fireTSDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
res9: fireTSDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


### Question 4
**Quels sont les types d'appels les plus courants?**

In [63]:
//Reponse 4
fireTSDF.groupBy($"CallType").count.orderBy(desc("count")).show(false)

+-------------------------------+------+
|CallType                       |count |
+-------------------------------+------+
|Medical Incident               |113794|
|Structure Fire                 |23319 |
|Alarms                         |19406 |
|Traffic Collision              |7013  |
|Citizen Assist / Service Call  |2524  |
|Other                          |2166  |
|Outside Fire                   |2094  |
|Vehicle Fire                   |854   |
|Gas Leak (Natural and LP Gases)|764   |
|Water Rescue                   |755   |
|Odor (Strange / Unknown)       |490   |
|Electrical Hazard              |482   |
|Elevator / Escalator Rescue    |453   |
|Smoke Investigation (Outside)  |391   |
|Fuel Spill                     |193   |
|HazMat                         |124   |
|Industrial Accidents           |94    |
|Explosion                      |89    |
|Train / Rail Incident          |57    |
|Aircraft Emergency             |36    |
+-------------------------------+------+
only showing top

### Question 5-a
**Quels sont boites postaux rencontrés dans les appels les plus courants?**

In [67]:
//Reponse 5-a
fireTSDF.groupBy($"Zipcode", $"CallType").count.orderBy(desc("count")).show(false)

+-------+----------------+-----+
|Zipcode|CallType        |count|
+-------+----------------+-----+
|94102  |Medical Incident|16130|
|94103  |Medical Incident|14775|
|94110  |Medical Incident|9995 |
|94109  |Medical Incident|9479 |
|94124  |Medical Incident|5885 |
|94112  |Medical Incident|5630 |
|94115  |Medical Incident|4785 |
|94122  |Medical Incident|4323 |
|94107  |Medical Incident|4284 |
|94133  |Medical Incident|3977 |
|94117  |Medical Incident|3522 |
|94134  |Medical Incident|3437 |
|94114  |Medical Incident|3225 |
|94118  |Medical Incident|3104 |
|94121  |Medical Incident|2953 |
|94116  |Medical Incident|2738 |
|94132  |Medical Incident|2594 |
|94110  |Structure Fire  |2267 |
|94105  |Medical Incident|2258 |
|94102  |Structure Fire  |2229 |
+-------+----------------+-----+
only showing top 20 rows



### Question 5-b
**Quels sont les quartiers de San Francisco dont les codes postaux sont 94102 et 94103?**

In [14]:
//Reponse 5-b
fireTSDF
    .groupBy($"Neighborhood", $"City", $"Zipcode")
    .count
    .filter($"City" === "San Francisco" and $"Zipcode" === 94102)
    .show(false)

+------------------------------+-------------+-------+-----+
|Neighborhood                  |City         |Zipcode|count|
+------------------------------+-------------+-------+-----+
|Tenderloin                    |San Francisco|94102  |5269 |
|Nob Hill                      |San Francisco|94102  |83   |
|Hayes Valley                  |San Francisco|94102  |877  |
|South of Market               |San Francisco|94102  |54   |
|Financial District/South Beach|San Francisco|94102  |149  |
|Western Addition              |San Francisco|94102  |281  |
+------------------------------+-------------+-------+-----+



In [15]:
fireTSDF
    .groupBy($"Neighborhood", $"City", $"Zipcode")
    .count
    .filter($"City" === "San Francisco" and $"Zipcode" === 94103 )
    .show(false)

+------------------------------+-------------+-------+-----+
|Neighborhood                  |City         |Zipcode|count|
+------------------------------+-------------+-------+-----+
|Hayes Valley                  |San Francisco|94103  |23   |
|Tenderloin                    |San Francisco|94103  |107  |
|South of Market               |San Francisco|94103  |4451 |
|Castro/Upper Market           |San Francisco|94103  |18   |
|Mission Bay                   |San Francisco|94103  |63   |
|Mission                       |San Francisco|94103  |1727 |
|Financial District/South Beach|San Francisco|94103  |361  |
|Potrero Hill                  |San Francisco|94103  |3    |
+------------------------------+-------------+-------+-----+



### Question 6
**Determiner le nombre total d'appels, ainsi que la moyenne, le minimum et le maximum du temps de réponse des appels?**

In [18]:
// Le nombre total d'appels
fireTSDF.select($"CallNumber").count

res12: Long = 175296


Les fonctions ***mean()***, ***min()*** et ***max()*** nous permettent respectivement de calculer la moyenne, le minimum et le maximum de la colonne **ResponseDelayedinMins**.

In [42]:
// La moyenne du temps de réponse des appels
fireTSDF.select(mean("ResponseDelayedinMins")).show(false)

+--------------------------+
|avg(ResponseDelayedinMins)|
+--------------------------+
|3.892364154521585         |
+--------------------------+



In [38]:
// Le minimum du temps de réponse des appels
fireTSDF.select(min("ResponseDelayedinMins")).show(false)

+--------------------------+
|min(ResponseDelayedinMins)|
+--------------------------+
|0.016666668               |
+--------------------------+



In [39]:
// Le maximum du temps de réponse des appels
fireTSDF.select(max("ResponseDelayedinMins")).show(false)

+--------------------------+
|max(ResponseDelayedinMins)|
+--------------------------+
|1844.55                   |
+--------------------------+



### Question 7-a
**Combien d'années distinctes trouve t-on dans ce Dataset?**  
Dans ce dataset nous avons des données comprises entre 2000-2018. Vous pouvez utilisez la fonction Spark `year()` pour les dates en Timestamp

In [77]:
//Reponse 7-a
fireTSDF.select(year($"OnWatchDate")).na.drop.distinct.count

res71: Long = 19


### Question 7-b
**Quelle semaine de l'année 2018 a eu le plus d'appels d'incendie?**

In [147]:
//Reponse 7-b
fireTSDF
    .groupBy($"OnWatchDate", $"CallNumber")
    .count
    .filter(year($"OnWatchDate") === "2018")
    .orderBy(desc("count"))
    .show(false)


+-------------------+----------+-----+
|OnWatchDate        |CallNumber|count|
+-------------------+----------+-----+
|2018-03-14 00:00:00|180732905 |4    |
|2018-03-26 00:00:00|180853836 |3    |
|2018-07-04 00:00:00|181852526 |3    |
|2018-04-21 00:00:00|181111911 |3    |
|2018-10-10 00:00:00|182840147 |3    |
|2018-11-01 00:00:00|183060064 |3    |
|2018-01-01 00:00:00|180013654 |3    |
|2018-02-19 00:00:00|180503139 |3    |
|2018-03-31 00:00:00|180902309 |2    |
|2018-05-04 00:00:00|181242198 |2    |
|2018-10-23 00:00:00|182963609 |2    |
|2018-07-10 00:00:00|181912524 |2    |
|2018-06-11 00:00:00|181630424 |2    |
|2018-08-02 00:00:00|182142756 |2    |
|2018-01-25 00:00:00|180252392 |2    |
|2018-02-07 00:00:00|180382638 |2    |
|2018-02-13 00:00:00|180442475 |2    |
|2018-09-16 00:00:00|182600557 |2    |
|2018-02-18 00:00:00|180492998 |2    |
|2018-05-01 00:00:00|181211554 |2    |
+-------------------+----------+-----+
only showing top 20 rows



### Question 8
**Quels sont les quartiers de San Francisco qui ont connu le pire temps de réponse en 2018?**

In [69]:
//Reponse 8
fireTSDF
    .groupBy($"Neighborhood", $"City", $"ResponseDelayedinMins")
    .count
    .filter($"City" === "San Francisco")
    .orderBy("ResponseDelayedinMins")
    .show(false)

+--------------------------+-------------+---------------------+-----+
|Neighborhood              |City         |ResponseDelayedinMins|count|
+--------------------------+-------------+---------------------+-----+
|Tenderloin                |San Francisco|0.06666667           |1    |
|South of Market           |San Francisco|0.06666667           |1    |
|Nob Hill                  |San Francisco|0.083333336          |1    |
|Mission Bay               |San Francisco|0.11666667           |1    |
|Tenderloin                |San Francisco|0.13333334           |1    |
|Mission Bay               |San Francisco|0.15                 |1    |
|Pacific Heights           |San Francisco|0.15                 |1    |
|McLaren Park              |San Francisco|0.16666667           |1    |
|Nob Hill                  |San Francisco|0.16666667           |1    |
|Castro/Upper Market       |San Francisco|0.16666667           |1    |
|Tenderloin                |San Francisco|0.16666667           |1    |
|Missi

### Question 9

**Comment stocker les données du Dataframe sous format de fichiers Parquet?**

In [151]:
//Reponse 9
fireTSDF.
    write.
    mode("overwrite").
    option("compression", "none").
    parquet("datasets/training/retail_db/fireTSDF")

### Question 10
**Comment relire les données stockée en format Parquet?**

In [154]:
//Reponse 10
spark.read.parquet("datasets/training/retail_db/fireTSDF").printSchema

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 |-- SupervisorDistrict: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Location: string (nullable =

## FIN